## 1. Libraries

In [1]:
import pandas as pd
import csv
import calendar
from datetime import datetime, timedelta

## 2. Loading the data

In [2]:
crimes_general = pd.read_csv(fr'C:\Users\Massieu\Desktop\Chicago\Crimes.csv', sep = ',')
districts = pd.read_excel(r'C:\Users\Massieu\Desktop\Chicago\Districts.xlsx')
crimes_general.head()

,Date,Block,Crime_Type,Location,Coordinates,District_Number,Arrest,Domestic
0,2021-12-31 12:55:00 UTC,020XX W NORTH AVE,BATTERY,BAR OR TAVERN,"(41.910470499, -87.678366829)",14.0,False,False
1,2021-12-31 12:49:00 UTC,075XX S SEELEY AVE,DECEPTIVE PRACTICE,RESIDENCE,"(41.756418241, -87.674522275)",6.0,False,False
2,2021-12-31 12:46:00 UTC,0000X W 79TH ST,ASSAULT,COMMERCIAL / BUSINESS OFFICE,"(41.75093383, -87.626288377)",6.0,False,False
3,2021-12-31 12:45:00 UTC,045XX S DREXEL BLVD,ASSAULT,APARTMENT,"(41.812412001, -87.604432386)",2.0,False,False
4,2021-12-31 12:45:00 UTC,085XX S ASHLAND AVE,CONCEALED CARRY LICENSE VIOLATION,STREET,"(41.738569281, -87.663137439)",6.0,True,False


## 3. Cleaning the data

In [3]:
# Data types, number of columns and rows, amount of null values inside the Dataframe
crimes_general.info(),crimes_general.isnull().sum(), crimes_general.isnull().sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668280 entries, 0 to 2668279
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Date             object 
 1   Block            object 
 2   Crime_Type       object 
 3   Location         object 
 4   Coordinates      object 
 5   District_Number  float64
 6   Arrest           bool   
 7   Domestic         bool   
dtypes: bool(2), float64(1), object(5)
memory usage: 127.2+ MB


(None,
 Date                   0
 Block                  0
 Crime_Type             0
 Location            8312
 Coordinates        28341
 District_Number        1
 Arrest                 0
 Domestic               0
 dtype: int64,
 36654)

In [4]:
# Removing undesired rows
crimes_general.dropna(subset = ['Location'], axis = 0, how = 'any', inplace = True)
crimes_general.dropna(subset = ['Coordinates'], axis = 0, how = 'any', inplace = True) 
crimes_general.dropna(subset = ['District_Number'], axis = 0, how = 'any', inplace = True)
crimes_general.isnull().sum()

Date               0
Block              0
Crime_Type         0
Location           0
Coordinates        0
District_Number    0
Arrest             0
Domestic           0
dtype: int64

In [5]:
crimes_general['District_Number'] = crimes_general['District_Number'].astype(int) # Changing data type
crimes_general[['Latitude', 'Longitude']] = crimes_general['Coordinates'].str.split(",", expand = True) # Splitting 'location' column
crimes_general['Latitude'] = crimes_general['Latitude'].str.replace('(',"") # Replacing values
crimes_general['Longitude'] = crimes_general['Longitude'].str.replace(')',"") # Replacing values
crimes_general['Crime_Type'] = crimes_general['Crime_Type'].str.replace('NON - CRIMINAL', 'NON-CRIMINAL') # Replacing values
crimes_general[['Latitude', 'Longitude']] = crimes_general[['Latitude', 'Longitude']].astype(float) # Changing data type
crimes_general.drop('Coordinates', axis = 1, inplace = True) # dropping 'location' column
crimes_general.info()

C:\Users\Massieu\anaconda3\Anaconda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Massieu\anaconda3\Anaconda\envs\myenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2634429 entries, 0 to 2668279
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Date             object 
 1   Block            object 
 2   Crime_Type       object 
 3   Location         object 
 4   District_Number  int32  
 5   Arrest           bool   
 6   Domestic         bool   
 7   Latitude         float64
 8   Longitude        float64
dtypes: bool(2), float64(2), int32(1), object(4)
memory usage: 155.8+ MB


In [6]:
crimes = pd.merge(crimes_general, districts, on = 'District_Number')
crimes.head()

,Date,Block,Crime_Type,Location,District_Number,Arrest,Domestic,Latitude,Longitude,District_Name
0,2021-12-31 12:55:00 UTC,020XX W NORTH AVE,BATTERY,BAR OR TAVERN,14,False,False,41.910470,-87.678367,Shakespeare
1,2021-12-31 12:00:00 UTC,029XX W MC LEAN AVE,THEFT,APARTMENT,14,False,False,41.918509,-87.700544,Shakespeare
2,2021-12-31 11:45:00 UTC,021XX N MILWAUKEE AVE,BATTERY,BAR OR TAVERN,14,False,False,41.920132,-87.692822,Shakespeare
3,2021-12-31 11:25:00 UTC,027XX N MILWAUKEE AVE,BATTERY,RESTAURANT,14,True,False,41.930381,-87.709555,Shakespeare
4,2021-12-31 10:00:00 UTC,033XX W LE MOYNE ST,THEFT,RESIDENCE - YARD (FRONT / BACK),14,False,False,41.908244,-87.710244,Shakespeare


In [7]:
crimes.drop_duplicates(subset = ['Date', 'Block', 'Crime_Type','Latitude', 'Longitude'], inplace = True)
crimes.head()

,Date,Block,Crime_Type,Location,District_Number,Arrest,Domestic,Latitude,Longitude,District_Name
0,2021-12-31 12:55:00 UTC,020XX W NORTH AVE,BATTERY,BAR OR TAVERN,14,False,False,41.910470,-87.678367,Shakespeare
1,2021-12-31 12:00:00 UTC,029XX W MC LEAN AVE,THEFT,APARTMENT,14,False,False,41.918509,-87.700544,Shakespeare
2,2021-12-31 11:45:00 UTC,021XX N MILWAUKEE AVE,BATTERY,BAR OR TAVERN,14,False,False,41.920132,-87.692822,Shakespeare
3,2021-12-31 11:25:00 UTC,027XX N MILWAUKEE AVE,BATTERY,RESTAURANT,14,True,False,41.930381,-87.709555,Shakespeare
4,2021-12-31 10:00:00 UTC,033XX W LE MOYNE ST,THEFT,RESIDENCE - YARD (FRONT / BACK),14,False,False,41.908244,-87.710244,Shakespeare


In [8]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2627120 entries, 0 to 2634353
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Date             object 
 1   Block            object 
 2   Crime_Type       object 
 3   Location         object 
 4   District_Number  int32  
 5   Arrest           bool   
 6   Domestic         bool   
 7   Latitude         float64
 8   Longitude        float64
 9   District_Name    object 
dtypes: bool(2), float64(2), int32(1), object(5)
memory usage: 175.4+ MB


## 4. Exporting it

In [9]:
crimes.to_csv(r'C:\Users\Massieu\Desktop\Chicago\Crimes_worked.csv', index = False)